# CONFLUENCE Tutorial - 9: NorSWE Large Sample Study (Snow Observation Network)

## Introduction
This tutorial extends our large sample studies approach to focus specifically on snow hydrology validation using the NorSWE (Northern Hemisphere Snow Water Equivalent) dataset. Building on the multi-site analysis framework demonstrated with FLUXNET, we now apply CONFLUENCE to systematically evaluate snow modeling performance across a network of snow observation stations throughout the northern hemisphere.

## NorSWE: A Critical Snow Observation Network
The NorSWE dataset represents one of the most comprehensive collections of snow observations available for hydrological model validation. The network provides extensive spatial coverage across snow-dominated regions of the Northern Hemisphere, with particularly dense coverage throughout Scandinavia, Finland, and Norway. Stations span elevation gradients from coastal lowlands to high mountain regions, capturing the full spectrum of snow climates including maritime, continental, and Arctic environments.
The observational richness of NorSWE includes direct measurements of Snow Water Equivalent (SWE) and complementary snow depth observations that provide critical information about snowpack structure. Many sites contain multi-decade records processed through standardized measurement protocols and quality control procedures, making them ideal for systematic model evaluation.

## Scientific Importance of Snow Validation
Snow processes represent some of the most challenging aspects of hydrological modeling due to their inherent physical complexity. The snowpack undergoes continuous phase transitions through freezing, melting, and sublimation processes, while complex energy balance interactions involving radiation, temperature, and wind drive temporal evolution. Internal layered structure develops through metamorphism and density changes, creating spatial variability that exhibits strong dependencies on elevation and topographic aspect.
The hydrological significance of snow extends far beyond its physical complexity. Snow functions as a natural seasonal reservoir in many regions, with snowmelt timing exerting primary control over peak flows and water availability. These processes exhibit high sensitivity to temperature changes, making them critical indicators of climate impacts. Additionally, extreme events such as snow-rain transitions and rain-on-snow scenarios pose significant challenges for both modeling and water resource management.

## Learning Outcomes
This tutorial demonstrates key capabilities for conducting snow-focused large sample studies through systematic application of CONFLUENCE. We show how to adapt CONFLUENCE configurations specifically for snow observation sites and focus analysis on snow accumulation and ablation periods. The tutorial covers multi-variable validation approaches that compare both SWE and snow depth simulations, while examining how model performance varies with elevation and assessing performance across different snow climate regimes.
The combination of CONFLUENCE's sophisticated snow modeling capabilities with NorSWE's comprehensive observation network provides a powerful framework for advancing snow science through systematic, large sample analysis.

## Step 1: Large Sample Snow Study Experimental Design and Site Selection
Transitioning from the FLUXNET energy balance focus to systematic snow hydrology validation, this step establishes the foundation for large sample snow modeling using the comprehensive NorSWE observation network. We demonstrate how CONFLUENCE's workflow efficiency enables systematic snow process evaluation across the full spectrum of northern hemisphere snow environments, from temperate mountain ranges to Arctic tundra.

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import yaml
from datetime import datetime
import seaborn as sns
import warnings

# Set up plotting style for snow visualization
plt.style.use('default')
sns.set_palette("coolwarm")
%matplotlib inline
confluence_path = Path('../').resolve()

# =============================================================================
# LARGE SAMPLE SNOW EXPERIMENTAL DESIGN CONFIGURATION
# =============================================================================

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data')  # ← Update this path
#CONFLUENCE_DATA_DIR = Path('/path/to/your/CONFLUENCE_data') 

# Load snow observation configuration template or create from base template
snow_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'
with open(snow_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for snow tutorial-specific settings
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'norswe_template',
    'EXPERIMENT_ID': 'run_1',
    'EXPERIMENT_TIME_START': '2018-01-01 01:00',
    'EXPERIMENT_TIME_END': '2018-03-31 23:00',  # Short for tutorial demonstration
}

config_dict.update(config_updates)

# Save snow configuration template
norswe_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_norswe_template.yaml'
with open(norswe_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"✅ NorSWE template configuration saved: {norswe_config_path}")

# =============================================================================
# LOAD AND EXAMINE NORSWE SNOW STATIONS DATASET
# =============================================================================

print(f"\n🌨️ Loading NorSWE Snow Station Database...")

# Load the NorSWE stations database
try:
    norswe_df = pd.read_csv('norswe_stations.csv')
    print(f"✅ Successfully loaded NorSWE database: {len(norswe_df)} snow stations available")
except FileNotFoundError:
    print(f"⚠️  NorSWE database not found, creating demonstration dataset...")
    
    # Create demonstration NorSWE dataset for tutorial
    np.random.seed(42)
    n_stations = 150
    
    # Generate realistic northern hemisphere snow station locations
    # Focus on snow-dominated regions: Scandinavia, Canada, Russia, Alps, etc.
    regions = [
        {'name': 'Scandinavia', 'lat_range': (55, 70), 'lon_range': (5, 30), 'n': 40},
        {'name': 'Canada', 'lat_range': (45, 70), 'lon_range': (-140, -60), 'n': 35},
        {'name': 'Russia/Siberia', 'lat_range': (50, 70), 'lon_range': (30, 150), 'n': 30},
        {'name': 'Alps/Mountains', 'lat_range': (45, 55), 'lon_range': (-10, 15), 'n': 25},
        {'name': 'Other Northern', 'lat_range': (40, 75), 'lon_range': (-180, 180), 'n': 20}
    ]
    
    stations_data = []
    station_id = 1
    
    for region in regions:
        for i in range(region['n']):
            lat = np.random.uniform(region['lat_range'][0], region['lat_range'][1])
            lon = np.random.uniform(region['lon_range'][0], region['lon_range'][1])
            
            # Elevation based on latitude and region (higher at mountains and northern latitudes)
            if region['name'] == 'Alps/Mountains':
                elevation = np.random.uniform(500, 3000)
            elif lat > 65:  # Arctic
                elevation = np.random.uniform(0, 800)
            else:
                elevation = np.random.uniform(0, 1500)
            
            # Data completeness (higher quality for more accessible sites)
            base_completeness = 90 - (lat - 40) * 0.8  # Lower completeness at higher latitudes
            swe_completeness = max(20, np.random.normal(base_completeness, 15))
            depth_completeness = max(15, np.random.normal(base_completeness + 5, 12))
            
            # Create station entry
            station = {
                'station_id': f"NOR_{station_id:04d}",
                'station_name': f"{region['name']}_Station_{i+1:03d}",
                'lat': round(lat, 4),
                'lon': round(lon, 4),
                'elevation': round(elevation, 0),
                'source': np.random.choice(['NorSWE', 'SYNOP', 'National_Met', 'Research'], p=[0.6, 0.2, 0.15, 0.05]),
                'swq_completeness': round(min(95, max(20, swe_completeness)), 1),
                'snd_completeness': round(min(95, max(15, depth_completeness)), 1),
            }
            
            # Add CONFLUENCE formatting
            buffer = 0.1
            station['BOUNDING_BOX_COORDS'] = f"{lat + buffer}/{lon - buffer}/{lat - buffer}/{lon + buffer}"
            station['POUR_POINT_COORDS'] = f"{lat}/{lon}"
            station['Watershed_Name'] = station['station_id'].replace(' ', '_')
            
            stations_data.append(station)
            station_id += 1
    
    norswe_df = pd.DataFrame(stations_data)
    
    # Save demonstration dataset
    norswe_df.to_csv('norswe_stations.csv', index=False)
    print(f"✅ Created demonstration NorSWE dataset: {len(norswe_df)} stations")

# Display basic dataset information
print(f"\n📊 Dataset Overview:")
print(f"  Total snow stations: {len(norswe_df)}")
print(f"  Columns: {len(norswe_df.columns)}")
print(f"  Column names: {', '.join(norswe_df.columns[:8])}...")

# =============================================================================
# EXTRACT SPATIAL COORDINATES AND SNOW-SPECIFIC ATTRIBUTES
# =============================================================================

print(f"\n🗺️  Extracting Snow Station Spatial Information...")

# Ensure coordinate columns exist
if 'latitude' not in norswe_df.columns and 'lat' in norswe_df.columns:
    norswe_df['latitude'] = norswe_df['lat']
if 'longitude' not in norswe_df.columns and 'lon' in norswe_df.columns:
    norswe_df['longitude'] = norswe_df['lon']

print(f"✅ Coordinate extraction successful")
print(f"  Latitude range: {norswe_df['latitude'].min():.1f}° to {norswe_df['latitude'].max():.1f}°N")
print(f"  Longitude range: {norswe_df['longitude'].min():.1f}° to {norswe_df['longitude'].max():.1f}°E")
print(f"  Elevation range: {norswe_df['elevation'].min():.0f}m to {norswe_df['elevation'].max():.0f}m")

# =============================================================================
# SNOW-SPECIFIC DATASET CHARACTERISTICS ANALYSIS
# =============================================================================

print(f"\n❄️ Analyzing Snow Dataset Characteristics...")

# Elevation-based snow climate zones
elevation_zones = [
    (0, 200, 'Coastal/Lowland'),
    (200, 500, 'Montane'),
    (500, 1000, 'Subalpine'),
    (1000, 1500, 'Alpine'),
    (1500, 10000, 'High Alpine')
]

norswe_df['elevation_zone'] = 'Unknown'
for min_elev, max_elev, zone_name in elevation_zones:
    mask = (norswe_df['elevation'] >= min_elev) & (norswe_df['elevation'] < max_elev)
    norswe_df.loc[mask, 'elevation_zone'] = zone_name

elevation_counts = norswe_df['elevation_zone'].value_counts()
print(f"  Elevation zones: {len(elevation_counts)}")
print(f"    Most common: {elevation_counts.index[0]} ({elevation_counts.iloc[0]} stations)")

# Latitude-based climate zones
latitude_zones = [
    (40, 50, 'Temperate'),
    (50, 60, 'Boreal'),
    (60, 70, 'Subarctic'),
    (70, 80, 'Arctic')
]

norswe_df['climate_zone'] = 'Unknown'
for min_lat, max_lat, zone_name in latitude_zones:
    mask = (norswe_df['latitude'] >= min_lat) & (norswe_df['latitude'] < max_lat)
    norswe_df.loc[mask, 'climate_zone'] = zone_name

climate_counts = norswe_df['climate_zone'].value_counts()
print(f"  Climate zones: {len(climate_counts)}")
print(f"    Most common: {climate_counts.index[0]} ({climate_counts.iloc[0]} stations)")

# Data source analysis
if 'source' in norswe_df.columns:
    source_counts = norswe_df['source'].value_counts()
    print(f"  Data sources: {len(source_counts)}")
    print(f"    Primary source: {source_counts.index[0]} ({source_counts.iloc[0]} stations)")

# Data quality analysis
if 'swq_completeness' in norswe_df.columns:
    swe_stats = norswe_df['swq_completeness'].describe()
    print(f"  SWE data completeness: {swe_stats['mean']:.1f}% ± {swe_stats['std']:.1f}%")
    
if 'snd_completeness' in norswe_df.columns:
    depth_stats = norswe_df['snd_completeness'].describe()
    print(f"  Snow depth completeness: {depth_stats['mean']:.1f}% ± {depth_stats['std']:.1f}%")

# =============================================================================
# SNOW DATASET VISUALIZATION
# =============================================================================

print(f"\n📈 Creating Snow Dataset Overview Visualization...")

# Create comprehensive snow dataset overview
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Northern Hemisphere snow station distribution map
ax1 = axes[0, 0]
scatter = ax1.scatter(norswe_df['longitude'], norswe_df['latitude'], 
                     c=norswe_df['elevation'], cmap='terrain', 
                     alpha=0.7, s=40, edgecolors='black', linewidth=0.5)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title(f'NorSWE Snow Station Distribution\n({len(norswe_df)} stations)')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-180, 180)
ax1.set_ylim(35, 80)  # Focus on northern hemisphere snow regions

# Add colorbar for elevation
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Elevation (m)')

# 2. Elevation zone distribution
ax2 = axes[0, 1]
elevation_counts = norswe_df['elevation_zone'].value_counts()
bars = ax2.bar(range(len(elevation_counts)), elevation_counts.values, 
               color='lightblue', alpha=0.7, edgecolor='black')
ax2.set_xticks(range(len(elevation_counts)))
ax2.set_xticklabels(elevation_counts.index, rotation=45, ha='right')
ax2.set_ylabel('Number of Stations')
ax2.set_title('Snow Stations by Elevation Zone')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, count in zip(bars, elevation_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            str(count), ha='center', va='bottom', fontweight='bold')

# 3. Climate zone distribution
ax3 = axes[0, 2]
climate_counts = norswe_df['climate_zone'].value_counts()
colors = ['gold', 'lightgreen', 'lightcoral', 'lightsteelblue']
bars = ax3.bar(range(len(climate_counts)), climate_counts.values, 
               color=colors[:len(climate_counts)], alpha=0.7, edgecolor='black')
ax3.set_xticks(range(len(climate_counts)))
ax3.set_xticklabels(climate_counts.index, rotation=45, ha='right')
ax3.set_ylabel('Number of Stations')
ax3.set_title('Snow Stations by Climate Zone')
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, count in zip(bars, climate_counts.values):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            str(count), ha='center', va='bottom', fontweight='bold')

# 4. Elevation vs Latitude relationship
ax4 = axes[1, 0]
ax4.scatter(norswe_df['latitude'], norswe_df['elevation'], 
           alpha=0.6, s=30, c='purple', edgecolors='black', linewidth=0.3)
ax4.set_xlabel('Latitude (°N)')
ax4.set_ylabel('Elevation (m)')
ax4.set_title('Snow Stations: Elevation vs Latitude')
ax4.grid(True, alpha=0.3)

# 5. Data quality assessment
ax5 = axes[1, 1]
if 'swq_completeness' in norswe_df.columns and 'snd_completeness' in norswe_df.columns:
    ax5.scatter(norswe_df['swq_completeness'], norswe_df['snd_completeness'], 
               alpha=0.6, s=30, c='orange', edgecolors='black', linewidth=0.3)
    ax5.set_xlabel('SWE Data Completeness (%)')
    ax5.set_ylabel('Snow Depth Data Completeness (%)')
    ax5.set_title('Snow Data Quality Assessment')
    ax5.grid(True, alpha=0.3)
    
    # Add quality threshold lines
    ax5.axvline(x=70, color='red', linestyle='--', alpha=0.7, label='70% threshold')
    ax5.axhline(y=70, color='red', linestyle='--', alpha=0.7)
    ax5.legend()

# 6. Data source distribution
ax6 = axes[1, 2]
if 'source' in norswe_df.columns:
    source_counts = norswe_df['source'].value_counts()
    wedges, texts, autotexts = ax6.pie(source_counts.values, labels=source_counts.index, 
                                      autopct='%1.1f%%', startangle=90,
                                      colors=['lightblue', 'lightgreen', 'lightcoral', 'gold'])
    ax6.set_title('Snow Stations by Data Source')
    
    # Make percentage text bold
    for autotext in autotexts:
        autotext.set_fontweight('bold')

plt.suptitle('NorSWE Snow Observation Network - Dataset Overview', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# =============================================================================
# SNOW-SPECIFIC SUMMARY STATISTICS
# =============================================================================

print(f"\n❄️ Snow Dataset Summary:")
print(f"  ❄️  Geographic coverage: {len(norswe_df)} snow stations across Northern Hemisphere")
print(f"  🌍 Latitudinal span: {norswe_df['latitude'].max() - norswe_df['latitude'].min():.0f}° (Arctic to temperate)")
print(f"  ⛰️  Elevation range: {norswe_df['elevation'].min():.0f}m to {norswe_df['elevation'].max():.0f}m")
print(f"  🌨️  Environmental diversity:")
print(f"    Climate zones: {len(norswe_df['climate_zone'].unique())} (Arctic to temperate)")
print(f"    Elevation zones: {len(norswe_df['elevation_zone'].unique())} (coastal to high alpine)")
if 'source' in norswe_df.columns:
    print(f"    Data sources: {len(norswe_df['source'].unique())}")

print(f"  📊 Data quality characteristics:")
if 'swq_completeness' in norswe_df.columns:
    high_quality_swe = len(norswe_df[norswe_df['swq_completeness'] >= 80])
    print(f"    High-quality SWE data (≥80%): {high_quality_swe} stations ({high_quality_swe/len(norswe_df)*100:.1f}%)")
if 'snd_completeness' in norswe_df.columns:
    high_quality_depth = len(norswe_df[norswe_df['snd_completeness'] >= 80])
    print(f"    High-quality depth data (≥80%): {high_quality_depth} stations ({high_quality_depth/len(norswe_df)*100:.1f}%)")

# Regional distribution summary
print(f"  🗺️  Regional distribution:")
for zone, count in climate_counts.head(4).items():
    print(f"    {zone}: {count} stations ({count/len(norswe_df)*100:.1f}%)")


## Step 2: Large Sample Snow Modeling Execution

Building on the NorSWE station selection from Step 1, we now execute systematic snow modeling across diverse northern hemisphere environments using the `run_watersheds_camelsspat.py`. This step demonstrates CONFLUENCE's capability for large sample snow process validation, scaling from individual snow physics to continental-scale comparative snow science.

### Snow Modeling at Scale: From Single Sites to Northern Hemisphere Analysis

**Traditional Snow Modeling Approach**: Manual site-by-site snow process studies typically involve individual location simulations with limited transferability and manual configuration for each elevation zone or climate region. This approach makes it difficult to distinguish universal snow physics from site-specific effects and provides limited ability to identify systematic snow model performance patterns.

**Large Sample Snow Modeling Approach**: Systematic validation across environmental gradients enables **automated snow configuration** across elevation, latitude, and climate zones through **parallel snow simulations** that leverage CONFLUENCE's computational efficiency. This approach provides **standardized snow validation** for direct performance comparison across sites, **multi-variable snow assessment** integrating SWE, snow depth, and seasonal dynamics, leading to **continental-scale insights** into snow process model performance patterns.


In [ ]:
def run_norswe_script_from_notebook():
    """
    Execute the run_norswe-2.py script from within the notebook
    """
    print(f"\n❄️ Executing NorSWE Large Sample Snow Processing Script...")
    
    script_path = "./run_norswe-2.py"
    
    if not Path(script_path).exists():
        print(f"❌ Script not found: {script_path}")
        return False
    
    print(f"   📝 Script location: {script_path}")
    print(f"   🎯 Target sites: {len(complete_stations)} NorSWE stations")
    print(f"   ⏰ Processing started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    try:
        # Prepare script arguments based on experiment configuration
        script_args = [
            'python', script_path,
            '--norswe_path', experiment_config['norswe_path'],
            '--template_config', experiment_config['template_config'],
            '--output_dir', experiment_config['output_dir'],
            '--config_dir', experiment_config['config_dir'],
            '--min_completeness', str(experiment_config['min_completeness']),
            '--max_stations', str(experiment_config['max_stations']),
            '--base_path', experiment_config['base_path']
        ]
        
        # Add optional year filtering
        if experiment_config.get('start_year'):
            script_args.extend(['--start_year', str(experiment_config['start_year'])])
        if experiment_config.get('end_year'):
            script_args.extend(['--end_year', str(experiment_config['end_year'])])
        
        # Add no_submit flag if specified
        if experiment_config.get('no_submit', False):
            script_args.append('--no_submit')
        
        print(f"   🔧 Script arguments: {' '.join(script_args[2:])}")
        
        # Create a process with input automation
        process = subprocess.Popen(
            script_args,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        # Send 'y' to confirm job submission when prompted (unless no_submit)
        if not experiment_config.get('no_submit', False):
            stdout, stderr = process.communicate(input='y\n')
        else:
            stdout, stderr = process.communicate()
        
        # Print the output
        if stdout:
            print("📋 Script Output:")
            for line in stdout.split('\n'):
                if line.strip():
                    print(f"   {line}")
        
        if stderr:
            print("⚠️  Script Warnings/Errors:")
            for line in stderr.split('\n'):
                if line.strip():
                    print(f"   {line}")
        
        if process.returncode == 0:
            print(f"✅ NorSWE processing script completed successfully")
            return True
        else:
            print(f"❌ Script failed with return code: {process.returncode}")
            return False
            
    except Exception as e:
        print(f"❌ Error running script: {e}")
        return False

# Execute the NorSWE processing script
script_success = run_norswe_script_from_notebook()

## Step 3: Multi-Site Snow Validation and Process Analysis
Having executed large sample snow modeling, we now demonstrate the analytical power that emerges from systematic multi-site snow validation using NorSWE observations. This step showcases comprehensive snow process evaluation, seasonal dynamics analysis, and elevation-climate performance assessment—the scientific breakthrough enabled by large sample snow hydrology methodology.


In [ ]:
def discover_completed_snow_domains():
    """
    Discover all completed NorSWE domain directories and their snow outputs
    """
    print(f"\n🔍 Discovering Completed NorSWE Snow Modeling Domains...")
    
    # Base data directory pattern
    base_path = Path(experiment_config['base_path'])
    domain_pattern = str(base_path / "domain_*")
    
    # Find all domain directories
    domain_dirs = glob.glob(domain_pattern)
    
    print(f"   📁 Found {len(domain_dirs)} total domain directories")
    
    completed_domains = []
    
    for domain_dir in domain_dirs:
        domain_path = Path(domain_dir)
        domain_name = domain_path.name.replace('domain_', '')
        
        # Check if this is a NorSWE domain (should match our selected stations)
        if any(domain_name in site for site in complete_stations['Watershed_Name'].values):
            
            # Check for key output files
            shapefile_path = domain_path / "shapefiles" / "catchment" / f"{domain_name}_HRUs.shp"
            simulation_dir = domain_path / "simulations"
            obs_dir = domain_path / "observations" / "snow" / "raw_data"
            
            domain_info = {
                'domain_name': domain_name,
                'domain_path': domain_path,
                'has_shapefile': shapefile_path.exists(),
                'shapefile_path': shapefile_path if shapefile_path.exists() else None,
                'has_simulations': simulation_dir.exists(),
                'simulation_path': simulation_dir if simulation_dir.exists() else None,
                'has_observations': obs_dir.exists(),
                'observation_path': obs_dir if obs_dir.exists() else None,
                'simulation_files': [],
                'swe_obs_file': None,
                'depth_obs_file': None
            }
            
            # Find simulation output files
            if simulation_dir.exists():
                nc_files = list(simulation_dir.glob("**/*.nc"))
                domain_info['simulation_files'] = nc_files
                domain_info['has_results'] = len(nc_files) > 0
            else:
                domain_info['has_results'] = False
            
            # Find observation files
            if obs_dir.exists():
                swe_files = list((obs_dir / "swe").glob("*.csv"))
                depth_files = list((obs_dir / "depth").glob("*.csv"))
                
                if swe_files:
                    domain_info['swe_obs_file'] = swe_files[0]
                if depth_files:
                    domain_info['depth_obs_file'] = depth_files[0]
            
            completed_domains.append(domain_info)
    
    print(f"   ❄️ NorSWE domains found: {len(completed_domains)}")
    print(f"   📊 Domains with shapefiles: {sum(1 for d in completed_domains if d['has_shapefile'])}")
    print(f"   📈 Domains with simulation results: {sum(1 for d in completed_domains if d['has_results'])}")
    print(f"   📋 Domains with observations: {sum(1 for d in completed_domains if d['has_observations'])}")
    
    return completed_domains

def create_snow_domain_overview_map(completed_domains):
    """
    Create an overview map showing all snow domain locations and their completion status
    """
    print(f"\n🗺️  Creating Snow Domain Overview Map...")
    
    # Create figure for overview map
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    # Map 1: Global overview with completion status (focus on Northern Hemisphere)
    ax1 = axes[0, 0]
    
    # Plot all selected sites
    ax1.scatter(complete_stations['lon'], complete_stations['lat'], 
               c='lightgray', alpha=0.5, s=30, label='Selected stations', marker='o')
    
    # Plot completed domains with different colors for different completion levels
    for domain in completed_domains:
        domain_name = domain['domain_name']
        
        # Find corresponding site in complete_stations
        site_row = complete_stations[complete_stations['Watershed_Name'] == domain_name]
        
        if not site_row.empty:
            lat = site_row['lat'].iloc[0]
            lon = site_row['lon'].iloc[0]
            
            # Color based on completion status
            if domain['has_results'] and domain['has_observations']:
                color = 'green'
                label = 'Complete with snow validation'
                marker = 's'
                size = 60
            elif domain['has_results']:
                color = 'orange' 
                label = 'Simulation complete'
                marker = '^'
                size = 50
            elif domain['has_observations']:
                color = 'blue'
                label = 'Observations only'
                marker = 'D'
                size = 40
            else:
                color = 'red'
                label = 'Processing started'
                marker = 'v'
                size = 30
            
            ax1.scatter(lon, lat, c=color, s=size, marker=marker, alpha=0.8,
                       edgecolors='black', linewidth=0.5)
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('NorSWE Snow Domain Processing Status Overview')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-180, 180)
    ax1.set_ylim(30, 85)  # Focus on Northern Hemisphere snow regions
    
    # Create custom legend
    legend_elements = [
        plt.scatter([], [], c='green', s=60, marker='s', label='Complete with validation'),
        plt.scatter([], [], c='orange', s=50, marker='^', label='Simulation complete'),
        plt.scatter([], [], c='blue', s=40, marker='D', label='Observations extracted'),
        plt.scatter([], [], c='red', s=30, marker='v', label='Processing started'),
        plt.scatter([], [], c='lightgray', s=30, marker='o', label='Selected stations')
    ]
    ax1.legend(handles=legend_elements, loc='lower left')
    
    # Map 2: Completion statistics by elevation bands
    ax2 = axes[0, 1]
    
    # Create elevation bands
    elevation_bands = [(0, 500), (500, 1000), (1000, 1500), (1500, 2000), (2000, 10000)]
    band_labels = ['0-500m', '500-1000m', '1000-1500m', '1500-2000m', '>2000m']
    
    elevation_completion = {}
    
    for domain in completed_domains:
        domain_name = domain['domain_name']
        site_row = complete_stations[complete_stations['Watershed_Name'] == domain_name]
        
        if not site_row.empty:
            elevation = site_row['elevation'].iloc[0]
            
            # Find elevation band
            for i, (min_elev, max_elev) in enumerate(elevation_bands):
                if min_elev <= elevation < max_elev:
                    band_label = band_labels[i]
                    
                    if band_label not in elevation_completion:
                        elevation_completion[band_label] = {'total': 0, 'complete': 0, 'partial': 0}
                    
                    elevation_completion[band_label]['total'] += 1
                    
                    if domain['has_results'] and domain['has_observations']:
                        elevation_completion[band_label]['complete'] += 1
                    elif domain['has_results'] or domain['has_observations']:
                        elevation_completion[band_label]['partial'] += 1
                    break
    
    # Create stacked bar chart
    if elevation_completion:
        bands = list(elevation_completion.keys())
        complete_counts = [elevation_completion[b]['complete'] for b in bands]
        partial_counts = [elevation_completion[b]['partial'] for b in bands]
        pending_counts = [elevation_completion[b]['total'] - 
                         elevation_completion[b]['complete'] - 
                         elevation_completion[b]['partial'] for b in bands]
        
        x_pos = range(len(bands))
        
        ax2.bar(x_pos, complete_counts, label='Complete', color='green', alpha=0.7)
        ax2.bar(x_pos, partial_counts, bottom=complete_counts, 
               label='Partial', color='orange', alpha=0.7)
        ax2.bar(x_pos, pending_counts, 
               bottom=[c+p for c,p in zip(complete_counts, partial_counts)], 
               label='Pending', color='red', alpha=0.7)
        
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(bands, rotation=45, ha='right')
        ax2.set_ylabel('Number of Sites')
        ax2.set_title('Processing Status by Elevation Band')
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
    
    # Map 3: Station elevation vs latitude
    ax3 = axes[1, 0]
    
    domain_elevations = []
    domain_latitudes = []
    
    for domain in completed_domains:
        domain_name = domain['domain_name']
        site_row = complete_stations[complete_stations['Watershed_Name'] == domain_name]
        
        if not site_row.empty:
            elevation = site_row['elevation'].iloc[0]
            latitude = site_row['lat'].iloc[0]
            domain_elevations.append(elevation)
            domain_latitudes.append(latitude)
            
            # Color code by completion status
            if domain['has_results'] and domain['has_observations']:
                color = 'green'
            elif domain['has_results']:
                color = 'orange'
            else:
                color = 'red'
            
            ax3.scatter(latitude, elevation, c=color, alpha=0.7, s=40, edgecolors='black', linewidth=0.5)
    
    ax3.set_xlabel('Latitude')
    ax3.set_ylabel('Elevation (m)')
    ax3.set_title('Station Distribution: Elevation vs Latitude')
    ax3.grid(True, alpha=0.3)
    
    # Map 4: Processing summary statistics
    ax4 = axes[1, 1]
    
    # Summary statistics
    total_selected = len(complete_stations)
    total_discovered = len(completed_domains)
    total_with_results = sum(1 for d in completed_domains if d['has_results'])
    total_with_obs = sum(1 for d in completed_domains if d['has_observations'])
    total_complete = sum(1 for d in completed_domains if d['has_results'] and d['has_observations'])
    
    categories = ['Selected', 'Processing\nStarted', 'Simulation\nComplete', 'Observations\nExtracted', 'Ready for\nValidation']
    counts = [total_selected, total_discovered, total_with_results, total_with_obs, total_complete]
    colors = ['lightblue', 'yellow', 'orange', 'cyan', 'green']
    
    bars = ax4.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    ax4.set_ylabel('Number of Sites')
    ax4.set_title('Snow Modeling Processing Progress')
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('NorSWE Large Sample Snow Study - Domain Overview', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save the overview map
    overview_path = experiment_dir / 'plots' / 'snow_domain_overview_map.png'
    overview_path.parent.mkdir(exist_ok=True)
    plt.savefig(overview_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Snow domain overview map saved: {overview_path}")
    
    return total_selected, total_discovered, total_with_results, total_with_obs, total_complete

def extract_snow_results_from_domains(completed_domains):
    """
    Extract snow simulation results (SWE and snow depth) from all completed domains
    """
    print(f"\n❄️ Extracting Snow Results from Completed Domains...")
    
    snow_results = []
    processing_summary = {
        'total_domains': len(completed_domains),
        'domains_with_results': 0,
        'domains_with_snow': 0,
        'failed_extractions': 0
    }
    
    for domain in completed_domains:
        if not domain['has_results']:
            continue
            
        domain_name = domain['domain_name']
        processing_summary['domains_with_results'] += 1
        
        try:
            print(f"   🔄 Processing {domain_name}...")
            
            # Find simulation output files
            nc_files = domain['simulation_files']
            
            # Look for daily or monthly output files
            daily_files = [f for f in nc_files if 'day' in f.name.lower()]
            monthly_files = [f for f in nc_files if 'month' in f.name.lower()]
            timestep_files = [f for f in nc_files if 'timestep' in f.name.lower()]
            
            output_file = None
            if daily_files:
                output_file = daily_files[0]
            elif timestep_files:
                output_file = timestep_files[0]
            elif monthly_files:
                output_file = monthly_files[0]
            elif nc_files:
                output_file = nc_files[0]  # Use any available file
            
            if output_file is None:
                print(f"     ❌ No suitable output files found")
                processing_summary['failed_extractions'] += 1
                continue
            
            # Load the netCDF file
            ds = xr.open_dataset(output_file)
            
            # Look for snow variables
            snow_vars = {}
            
            # Common SUMMA snow variable names
            if 'scalarSWE' in ds.data_vars:
                snow_vars['swe'] = 'scalarSWE'
            elif 'SWE' in ds.data_vars:
                snow_vars['swe'] = 'SWE'
            
            if 'scalarSnowDepth' in ds.data_vars:
                snow_vars['depth'] = 'scalarSnowDepth'
            elif 'snowDepth' in ds.data_vars:
                snow_vars['depth'] = 'snowDepth'
            elif 'snow_depth' in ds.data_vars:
                snow_vars['depth'] = 'snow_depth'
            
            if not snow_vars:
                print(f"     ⚠️  No snow variables found in {output_file.name}")
                available_vars = list(ds.data_vars.keys())
                print(f"     Available variables: {available_vars[:10]}...")
                processing_summary['failed_extractions'] += 1
                continue
            
            print(f"     ❄️ Using snow variables: {snow_vars}")
            
            # Extract snow data
            extracted_data = {}
            
            for var_type, var_name in snow_vars.items():
                snow_data = ds[var_name]
                
                # Handle multi-dimensional data (take spatial mean if needed)
                if len(snow_data.dims) > 1:
                    spatial_dims = [dim for dim in snow_data.dims if dim != 'time']
                    if spatial_dims:
                        snow_data = snow_data.mean(dim=spatial_dims)
                
                # Convert to pandas Series
                snow_series = snow_data.to_pandas()
                
                # Handle negative values and unit conversion
                if var_type == 'swe':
                    # SWE should be positive
                    snow_series = snow_series.abs()
                    # Convert from kg/m² to mm if needed (1 kg/m² = 1 mm)
                    # SUMMA typically outputs in kg/m²
                elif var_type == 'depth':
                    # Snow depth should be positive
                    snow_series = snow_series.abs()
                    # Convert from m to cm if needed
                    if snow_series.max() < 10:  # Assume meters
                        snow_series = snow_series * 100  # Convert to cm
                
                extracted_data[var_type] = snow_series
            
            # Get site information
            site_row = complete_stations[complete_stations['Watershed_Name'] == domain_name]
            
            if site_row.empty:
                print(f"     ⚠️  Site information not found for {domain_name}")
                continue
            
            # Calculate snow season statistics
            snow_stats = {}
            
            for var_type, series in extracted_data.items():
                # Basic statistics
                snow_stats[f'{var_type}_mean'] = series.mean()
                snow_stats[f'{var_type}_max'] = series.max()
                snow_stats[f'{var_type}_std'] = series.std()
                
                # Seasonal statistics
                if len(series) > 0:
                    # Find peak snow (maximum value)
                    peak_idx = series.idxmax()
                    snow_stats[f'{var_type}_peak_date'] = peak_idx
                    snow_stats[f'{var_type}_peak_value'] = series[peak_idx]
                    
                    # Snow season length (days with snow > threshold)
                    threshold = 10 if var_type == 'swe' else 5  # 10 mm SWE or 5 cm depth
                    snow_days = (series > threshold).sum()
                    snow_stats[f'{var_type}_season_length'] = snow_days
            
            # Store results
            result = {
                'domain_name': domain_name,
                'station_id': site_row['station_id'].iloc[0],
                'latitude': site_row['lat'].iloc[0],
                'longitude': site_row['lon'].iloc[0],
                'elevation': site_row['elevation'].iloc[0],
                'data_period': f"{extracted_data[list(extracted_data.keys())[0]].index.min()} to {extracted_data[list(extracted_data.keys())[0]].index.max()}",
                'data_points': len(extracted_data[list(extracted_data.keys())[0]]),
                'output_file': str(output_file)
            }
            
            # Add time series data
            result.update(extracted_data)
            
            # Add statistics
            result.update(snow_stats)
            
            snow_results.append(result)
            processing_summary['domains_with_snow'] += 1
            
            swe_info = f"{snow_stats.get('swe_mean', 0):.1f} mm (max: {snow_stats.get('swe_max', 0):.1f})" if 'swe' in extracted_data else "N/A"
            depth_info = f"{snow_stats.get('depth_mean', 0):.1f} cm (max: {snow_stats.get('depth_max', 0):.1f})" if 'depth' in extracted_data else "N/A"
            
            print(f"     ✅ Snow extracted - SWE: {swe_info}, Depth: {depth_info}")
            
        except Exception as e:
            print(f"     ❌ Error processing {domain_name}: {e}")
            processing_summary['failed_extractions'] += 1
    
    print(f"\n❄️ Snow Extraction Summary:")
    print(f"   Total domains: {processing_summary['total_domains']}")
    print(f"   Domains with results: {processing_summary['domains_with_results']}")
    print(f"   Successful snow extractions: {processing_summary['domains_with_snow']}")
    print(f"   Failed extractions: {processing_summary['failed_extractions']}")
    
    return snow_results, processing_summary

def load_norswe_observations(completed_domains):
    """
    Load NorSWE observation data for snow validation
    """
    print(f"\n📥 Loading NorSWE Snow Observation Data...")
    
    norswe_obs = {}
    obs_summary = {
        'sites_found': 0,
        'sites_with_swe': 0,
        'sites_with_depth': 0,
        'total_swe_observations': 0,
        'total_depth_observations': 0
    }
    
    # Look for extracted NorSWE observation data in domain directories
    for domain in completed_domains:
        if not domain['has_observations']:
            continue
            
        domain_name = domain['domain_name']
        
        try:
            print(f"   📊 Loading {domain_name}...")
            
            obs_summary['sites_found'] += 1
            site_obs = {}
            
            # Load SWE observations
            if domain['swe_obs_file']:
                swe_df = pd.read_csv(domain['swe_obs_file'])
                swe_df['time'] = pd.to_datetime(swe_df['time'])
                swe_df.set_index('time', inplace=True)
                
                swe_obs = swe_df['SWE_kg_m2'].dropna()
                
                if len(swe_obs) > 0:
                    site_obs['swe_timeseries'] = swe_obs
                    site_obs['swe_mean'] = swe_obs.mean()
                    site_obs['swe_max'] = swe_obs.max()
                    site_obs['swe_std'] = swe_obs.std()
                    
                    # Seasonal statistics
                    peak_idx = swe_obs.idxmax()
                    site_obs['swe_peak_date'] = peak_idx
                    site_obs['swe_peak_value'] = swe_obs[peak_idx]
                    
                    # Snow season length
                    snow_days = (swe_obs > 10).sum()  # Days with >10mm SWE
                    site_obs['swe_season_length'] = snow_days
                    
                    obs_summary['sites_with_swe'] += 1
                    obs_summary['total_swe_observations'] += len(swe_obs)
                    
                    print(f"     ❄️ SWE obs: {swe_obs.mean():.1f} ± {swe_obs.std():.1f} mm ({len(swe_obs)} points)")
            
            # Load snow depth observations  
            if domain['depth_obs_file']:
                depth_df = pd.read_csv(domain['depth_obs_file'])
                depth_df['time'] = pd.to_datetime(depth_df['time'])
                depth_df.set_index('time', inplace=True)
                
                depth_obs = depth_df['Depth_m'].dropna() * 100  # Convert m to cm
                
                if len(depth_obs) > 0:
                    site_obs['depth_timeseries'] = depth_obs
                    site_obs['depth_mean'] = depth_obs.mean()
                    site_obs['depth_max'] = depth_obs.max()
                    site_obs['depth_std'] = depth_obs.std()
                    
                    # Seasonal statistics
                    peak_idx = depth_obs.idxmax()
                    site_obs['depth_peak_date'] = peak_idx
                    site_obs['depth_peak_value'] = depth_obs[peak_idx]
                    
                    # Snow season length
                    snow_days = (depth_obs > 5).sum()  # Days with >5cm depth
                    site_obs['depth_season_length'] = snow_days
                    
                    obs_summary['sites_with_depth'] += 1
                    obs_summary['total_depth_observations'] += len(depth_obs)
                    
                    print(f"     📏 Depth obs: {depth_obs.mean():.1f} ± {depth_obs.std():.1f} cm ({len(depth_obs)} points)")
            
            # Add site metadata
            site_row = complete_stations[complete_stations['Watershed_Name'] == domain_name]
            if not site_row.empty:
                site_obs['latitude'] = site_row['lat'].iloc[0]
                site_obs['longitude'] = site_row['lon'].iloc[0]
                site_obs['elevation'] = site_row['elevation'].iloc[0]
                site_obs['station_id'] = site_row['station_id'].iloc[0]
                
                norswe_obs[domain_name] = site_obs
                
        except Exception as e:
            print(f"     ❌ Error loading {domain_name}: {e}")
    
    print(f"\n❄️ NorSWE Observation Summary:")
    print(f"   Sites with observation files: {obs_summary['sites_found']}")
    print(f"   Sites with SWE observations: {obs_summary['sites_with_swe']}")
    print(f"   Sites with depth observations: {obs_summary['sites_with_depth']}")
    print(f"   Total SWE observations: {obs_summary['total_swe_observations']}")
    print(f"   Total depth observations: {obs_summary['total_depth_observations']}")
    
    return norswe_obs, obs_summary

def create_snow_comparison_analysis(snow_results, norswe_obs):
    """
    Create comprehensive snow comparison analysis between simulated and observed snow
    """
    print(f"\n❄️ Creating Snow Comparison Analysis...")
    
    # Find sites with both simulated and observed data
    common_sites = []
    
    for sim_result in snow_results:
        domain_name = sim_result['domain_name']
        
        if domain_name in norswe_obs:
            # Align time periods for both SWE and snow depth
            comparisons = {}
            
            # SWE comparison
            if 'swe' in sim_result and 'swe_timeseries' in norswe_obs[domain_name]:
                sim_swe = sim_result['swe']
                obs_swe = norswe_obs[domain_name]['swe_timeseries']
                
                # Find common time period
                common_start = max(sim_swe.index.min(), obs_swe.index.min())
                common_end = min(sim_swe.index.max(), obs_swe.index.max())
                
                if common_start < common_end:
                    # Resample to daily and align
                    sim_daily = sim_swe.resample('D').mean().loc[common_start:common_end]
                    obs_daily = obs_swe.resample('D').mean().loc[common_start:common_end]
                    
                    # Remove NaN values
                    valid_mask = ~(sim_daily.isna() | obs_daily.isna())
                    sim_valid = sim_daily[valid_mask]
                    obs_valid = obs_daily[valid_mask]
                    
                    if len(sim_valid) > 30:  # Need minimum data for meaningful comparison
                        
                        # Calculate performance metrics
                        rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
                        bias = (sim_valid - obs_valid).mean()
                        mae = np.abs(obs_valid - sim_valid).mean()
                        
                        # Correlation
                        try:
                            correlation = obs_valid.corr(sim_valid)
                            if pd.isna(correlation):
                                correlation = 0.0
                        except:
                            correlation = 0.0
                        
                        # Nash-Sutcliffe Efficiency
                        if obs_valid.var() > 0:
                            nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
                        else:
                            nse = np.nan
                        
                        comparisons['swe'] = {
                            'sim_data': sim_valid,
                            'obs_data': obs_valid,
                            'rmse': rmse,
                            'bias': bias,
                            'mae': mae,
                            'correlation': correlation,
                            'nse': nse,
                            'n_points': len(sim_valid)
                        }
            
            # Snow depth comparison
            if 'depth' in sim_result and 'depth_timeseries' in norswe_obs[domain_name]:
                sim_depth = sim_result['depth']
                obs_depth = norswe_obs[domain_name]['depth_timeseries']
                
                # Find common time period
                common_start = max(sim_depth.index.min(), obs_depth.index.min())
                common_end = min(sim_depth.index.max(), obs_depth.index.max())
                
                if common_start < common_end:
                    # Resample to daily and align
                    sim_daily = sim_depth.resample('D').mean().loc[common_start:common_end]
                    obs_daily = obs_depth.resample('D').mean().loc[common_start:common_end]
                    
                    # Remove NaN values
                    valid_mask = ~(sim_daily.isna() | obs_daily.isna())
                    sim_valid = sim_daily[valid_mask]
                    obs_valid = obs_daily[valid_mask]
                    
                    if len(sim_valid) > 30:  # Need minimum data for meaningful comparison
                        
                        # Calculate performance metrics
                        rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
                        bias = (sim_valid - obs_valid).mean()
                        mae = np.abs(obs_valid - sim_valid).mean()
                        
                        # Correlation
                        try:
                            correlation = obs_valid.corr(sim_valid)
                            if pd.isna(correlation):
                                correlation = 0.0
                        except:
                            correlation = 0.0
                        
                        # Nash-Sutcliffe Efficiency
                        if obs_valid.var() > 0:
                            nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
                        else:
                            nse = np.nan
                        
                        comparisons['depth'] = {
                            'sim_data': sim_valid,
                            'obs_data': obs_valid,
                            'rmse': rmse,
                            'bias': bias,
                            'mae': mae,
                            'correlation': correlation,
                            'nse': nse,
                            'n_points': len(sim_valid)
                        }
            
            if comparisons:
                common_site = {
                    'domain_name': domain_name,
                    'latitude': sim_result['latitude'],
                    'longitude': sim_result['longitude'],
                    'elevation': sim_result['elevation'],
                    'station_id': sim_result['station_id'],
                    'comparisons': comparisons
                }
                
                common_sites.append(common_site)
                
                # Print summary
                comp_summary = []
                for var_type, comp_data in comparisons.items():
                    comp_summary.append(f"{var_type}: r={comp_data['correlation']:.3f}, RMSE={comp_data['rmse']:.2f}")
                
                print(f"   ✅ {domain_name}: {', '.join(comp_summary)} ({comp_data['n_points']} points)")
    
    print(f"\n❄️ Snow Comparison Summary:")
    print(f"   Sites with both sim and obs: {len(common_sites)}")
    
    if len(common_sites) == 0:
        print("   ⚠️  No sites with overlapping sim/obs data for comparison")
        return None
    
    # Create comprehensive snow comparison visualization
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # SWE scatter plot (top left)
    ax1 = axes[0, 0]
    
    swe_sites = [site for site in common_sites if 'swe' in site['comparisons']]
    
    if swe_sites:
        all_obs_swe = np.concatenate([site['comparisons']['swe']['obs_data'].values for site in swe_sites])
        all_sim_swe = np.concatenate([site['comparisons']['swe']['sim_data'].values for site in swe_sites])
        
        ax1.scatter(all_obs_swe, all_sim_swe, alpha=0.5, s=15, c='blue')
        
        # 1:1 line
        min_val = min(all_obs_swe.min(), all_sim_swe.min())
        max_val = max(all_obs_swe.max(), all_sim_swe.max())
        ax1.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 line')
        
        ax1.set_xlabel('Observed SWE (mm)')
        ax1.set_ylabel('Simulated SWE (mm)')
        ax1.set_title('SWE: Simulated vs Observed')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Add overall statistics
        overall_corr = np.corrcoef(all_obs_swe, all_sim_swe)[0,1]
        overall_rmse = np.sqrt(np.mean((all_obs_swe - all_sim_swe)**2))
        overall_bias = np.mean(all_sim_swe - all_obs_swe)
        
        stats_text = f'r = {overall_corr:.3f}\nRMSE = {overall_rmse:.1f}\nBias = {overall_bias:+.1f}'
        ax1.text(0.05, 0.95, stats_text, transform=ax1.transAxes,
                 bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Snow depth scatter plot (top middle)
    ax2 = axes[0, 1]
    
    depth_sites = [site for site in common_sites if 'depth' in site['comparisons']]
    
    if depth_sites:
        all_obs_depth = np.concatenate([site['comparisons']['depth']['obs_data'].values for site in depth_sites])
        all_sim_depth = np.concatenate([site['comparisons']['depth']['sim_data'].values for site in depth_sites])
        
        ax2.scatter(all_obs_depth, all_sim_depth, alpha=0.5, s=15, c='purple')
        
        # 1:1 line
        min_val = min(all_obs_depth.min(), all_sim_depth.min())
        max_val = max(all_obs_depth.max(), all_sim_depth.max())
        ax2.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 line')
        
        ax2.set_xlabel('Observed Snow Depth (cm)')
        ax2.set_ylabel('Simulated Snow Depth (cm)')
        ax2.set_title('Snow Depth: Simulated vs Observed')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Add overall statistics
        overall_corr = np.corrcoef(all_obs_depth, all_sim_depth)[0,1]
        overall_rmse = np.sqrt(np.mean((all_obs_depth - all_sim_depth)**2))
        overall_bias = np.mean(all_sim_depth - all_obs_depth)
        
        stats_text = f'r = {overall_corr:.3f}\nRMSE = {overall_rmse:.1f}\nBias = {overall_bias:+.1f}'
        ax2.text(0.05, 0.95, stats_text, transform=ax2.transAxes,
                 bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Performance vs elevation (top right)
    ax3 = axes[0, 2]
    
    elevations = [site['elevation'] for site in common_sites if 'swe' in site['comparisons']]
    swe_correlations = [site['comparisons']['swe']['correlation'] for site in common_sites if 'swe' in site['comparisons']]
    
    if elevations and swe_correlations:
        ax3.scatter(elevations, swe_correlations, alpha=0.7, s=40, c='green')
        ax3.set_xlabel('Elevation (m)')
        ax3.set_ylabel('SWE Correlation')
        ax3.set_title('SWE Performance vs Elevation')
        ax3.grid(True, alpha=0.3)
        ax3.set_ylim(0, 1)
    
    # SWE bias distribution (bottom left)
    ax4 = axes[1, 0]
    
    if swe_sites:
        swe_biases = [site['comparisons']['swe']['bias'] for site in swe_sites]
        ax4.hist(swe_biases, bins=15, color='lightblue', alpha=0.7, edgecolor='black')
        ax4.axvline(x=0, color='red', linestyle='--', label='Zero bias')
        ax4.set_xlabel('SWE Bias (mm)')
        ax4.set_ylabel('Number of Sites')
        ax4.set_title('Distribution of SWE Bias')
        ax4.legend()
        ax4.grid(True, alpha=0.3, axis='y')
    
    # Snow depth bias distribution (bottom middle)
    ax5 = axes[1, 1]
    
    if depth_sites:
        depth_biases = [site['comparisons']['depth']['bias'] for site in depth_sites]
        ax5.hist(depth_biases, bins=15, color='lightcoral', alpha=0.7, edgecolor='black')
        ax5.axvline(x=0, color='red', linestyle='--', label='Zero bias')
        ax5.set_xlabel('Snow Depth Bias (cm)')
        ax5.set_ylabel('Number of Sites')
        ax5.set_title('Distribution of Snow Depth Bias')
        ax5.legend()
        ax5.grid(True, alpha=0.3, axis='y')
    
    # Performance by latitude (bottom right)
    ax6 = axes[1, 2]
    
    latitudes = [site['latitude'] for site in common_sites if 'swe' in site['comparisons']]
    swe_rmses = [site['comparisons']['swe']['rmse'] for site in common_sites if 'swe' in site['comparisons']]
    
    if latitudes and swe_rmses:
        ax6.scatter(latitudes, swe_rmses, alpha=0.7, s=40, c='orange')
        ax6.set_xlabel('Latitude')
        ax6.set_ylabel('SWE RMSE (mm)')
        ax6.set_title('SWE Performance vs Latitude')
        ax6.grid(True, alpha=0.3)
    
    plt.suptitle('NorSWE Large Sample Snow Comparison Analysis', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save comparison plot
    comparison_path = experiment_dir / 'plots' / 'snow_comparison_analysis.png'
    plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Snow comparison analysis saved: {comparison_path}")
    
    # Create spatial performance map
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Map 1: SWE correlation map
    ax1 = axes[0]
    
    lats = [site['latitude'] for site in common_sites if 'swe' in site['comparisons']]
    lons = [site['longitude'] for site in common_sites if 'swe' in site['comparisons']]
    corrs = [site['comparisons']['swe']['correlation'] for site in common_sites if 'swe' in site['comparisons']]
    
    if lats and lons and corrs:
        scatter1 = ax1.scatter(lons, lats, c=corrs, cmap='RdYlBu', s=80, 
                              vmin=0, vmax=1, edgecolors='black', linewidth=0.5)
        
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.set_title('Snow Model Performance: SWE Correlation')
        ax1.grid(True, alpha=0.3)
        ax1.set_xlim(-180, 180)
        ax1.set_ylim(30, 85)  # Northern Hemisphere focus
        
        # Add colorbar
        cbar1 = plt.colorbar(scatter1, ax=ax1)
        cbar1.set_label('SWE Correlation')
    
    # Map 2: SWE bias map
    ax2 = axes[1]
    
    biases = [site['comparisons']['swe']['bias'] for site in common_sites if 'swe' in site['comparisons']]
    
    if lats and lons and biases:
        max_abs_bias = max(abs(min(biases)), abs(max(biases)))
        
        scatter2 = ax2.scatter(lons, lats, c=biases, cmap='RdBu_r', s=80,
                              vmin=-max_abs_bias, vmax=max_abs_bias, 
                              edgecolors='black', linewidth=0.5)
        
        ax2.set_xlabel('Longitude')
        ax2.set_ylabel('Latitude')
        ax2.set_title('Snow Model Performance: SWE Bias (Sim - Obs)')
        ax2.grid(True, alpha=0.3)
        ax2.set_xlim(-180, 180)
        ax2.set_ylim(30, 85)
        
        # Add colorbar
        cbar2 = plt.colorbar(scatter2, ax=ax2)
        cbar2.set_label('SWE Bias (mm)')
    
    plt.suptitle('NorSWE Large Sample Snow Performance - Spatial Distribution', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save spatial analysis
    spatial_path = experiment_dir / 'plots' / 'snow_spatial_performance.png'
    plt.savefig(spatial_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Snow spatial performance map saved: {spatial_path}")
    
    return common_sites

# Execute Step 3 Analysis
print(f"\n🔍 Step 3.1: Snow Domain Discovery and Overview")

# Discover completed domains
completed_domains = discover_completed_snow_domains()

# Create domain overview map
total_selected, total_discovered, total_with_results, total_with_obs, total_complete = create_snow_domain_overview_map(completed_domains)

print(f"\n❄️ Step 3.2: Snow Results Extraction")

# Extract snow results from simulations
snow_results, snow_processing_summary = extract_snow_results_from_domains(completed_domains)

# Load NorSWE observations
norswe_obs, obs_summary = load_norswe_observations(completed_domains)

print(f"\n❄️ Step 3.3: Snow Comparison Analysis")

# Create snow comparison analysis
if snow_results and norswe_obs:
    common_sites = create_snow_comparison_analysis(snow_results, norswe_obs)
else:
    print("   ⚠️  Insufficient data for snow comparison analysis")
    common_sites = None

# Create final summary report
print(f"\n📋 Creating Final NorSWE Snow Study Summary Report...")

summary_report_path = experiment_dir / 'reports' / 'norswe_final_report.txt'
summary_report_path.parent.mkdir(exist_ok=True)

with open(summary_report_path, 'w') as f:
    f.write("NorSWE Large Sample Snow Study - Final Analysis Report\n")
    f.write("=" * 58 + "\n\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("PROCESSING SUMMARY:\n")
    f.write(f"  Sites selected for analysis: {total_selected}\n")
    f.write(f"  Processing initiated: {total_discovered}\n")
    f.write(f"  Simulation results available: {total_with_results}\n")
    f.write(f"  Observations extracted: {total_with_obs}\n")
    f.write(f"  Complete snow validation: {total_complete}\n")
    f.write(f"  Snow extractions successful: {snow_processing_summary['domains_with_snow']}\n")
    f.write(f"  NorSWE observations available: {obs_summary['sites_with_swe']}\n")
    
    if common_sites:
        f.write(f"  Sites with sim/obs comparison: {len(common_sites)}\n\n")
        
        # SWE performance summary
        swe_sites = [site for site in common_sites if 'swe' in site['comparisons']]
        if swe_sites:
            swe_correlations = [site['comparisons']['swe']['correlation'] for site in swe_sites]
            swe_rmses = [site['comparisons']['swe']['rmse'] for site in swe_sites]
            swe_biases = [site['comparisons']['swe']['bias'] for site in swe_sites]
            
            f.write("SWE PERFORMANCE SUMMARY:\n")
            f.write(f"  Mean correlation: {np.mean(swe_correlations):.3f} ± {np.std(swe_correlations):.3f}\n")
            f.write(f"  Mean RMSE: {np.mean(swe_rmses):.1f} ± {np.std(swe_rmses):.1f} mm\n")
            f.write(f"  Mean bias: {np.mean(swe_biases):+.1f} ± {np.std(swe_biases):.1f} mm\n\n")
        
        # Snow depth performance summary
        depth_sites = [site for site in common_sites if 'depth' in site['comparisons']]
        if depth_sites:
            depth_correlations = [site['comparisons']['depth']['correlation'] for site in depth_sites]
            depth_rmses = [site['comparisons']['depth']['rmse'] for site in depth_sites]
            depth_biases = [site['comparisons']['depth']['bias'] for site in depth_sites]
            
            f.write("SNOW DEPTH PERFORMANCE SUMMARY:\n")
            f.write(f"  Mean correlation: {np.mean(depth_correlations):.3f} ± {np.std(depth_correlations):.3f}\n")
            f.write(f"  Mean RMSE: {np.mean(depth_rmses):.1f} ± {np.std(depth_rmses):.1f} cm\n")
            f.write(f"  Mean bias: {np.mean(depth_biases):+.1f} ± {np.std(depth_biases):.1f} cm\n\n")
        
        f.write("BEST PERFORMING SITES (by SWE correlation):\n")
        if swe_sites:
            sorted_sites = sorted(swe_sites, key=lambda x: x['comparisons']['swe']['correlation'], reverse=True)
            for i, site in enumerate(sorted_sites[:5]):
                f.write(f"  {i+1}. {site['domain_name']}: r={site['comparisons']['swe']['correlation']:.3f}, RMSE={site['comparisons']['swe']['rmse']:.1f} mm\n")

print(f"✅ Final summary report saved: {summary_report_path}")

print(f"\n🎉 Step 3 Complete: NorSWE Snow Validation Analysis")
print(f"   📁 Results saved to: {experiment_dir}")
print(f"   ❄️ Snow domain overview: {total_complete}/{total_selected} sites with complete validation")

if common_sites:
    swe_sites = [site for site in common_sites if 'swe' in site['comparisons']]
    depth_sites = [site for site in common_sites if 'depth' in site['comparisons']]
    
    if swe_sites:
        swe_correlations = [site['comparisons']['swe']['correlation'] for site in swe_sites]
        print(f"   📊 SWE analysis: {len(swe_sites)} sites with sim/obs comparison")
        print(f"   📈 SWE performance: Mean r = {np.mean(swe_correlations):.3f}")
    
    if depth_sites:
        depth_correlations = [site['comparisons']['depth']['correlation'] for site in depth_sites]
        print(f"   📏 Depth analysis: {len(depth_sites)} sites with sim/obs comparison")
        print(f"   📈 Depth performance: Mean r = {np.mean(depth_correlations):.3f}")
else:
    print(f"   📈 Performance: Awaiting more simulation results")

print(f"\n✅ Large Sample NorSWE Snow Analysis Complete!")
print(f"   ❄️ Multi-site snow hydrology validation achieved")
print(f"   📊 Statistical patterns identified across elevation and climate gradients")

## Tutorial Summary: Large Sample Snow Hydrology with NorSWE

This tutorial demonstrated the power of large sample snow modeling through systematic validation across northern hemisphere environmental gradients. Using the comprehensive NorSWE observation network, we successfully scaled from individual snow physics simulations to continental-scale comparative snow science.

**Key Accomplishments:**
- **Multi-site snow evaluation** across elevation zones from coastal lowlands to high alpine environments
- **Climate gradient analysis** spanning temperate, boreal, and Arctic snow regimes  
- **Systematic snow process evaluation** comparing SWE and snow depth simulations against observations
- **Performance pattern identification** revealing how snow model accuracy varies with elevation, latitude, and climate

**Scientific Insights Gained:**
The large sample approach revealed systematic patterns in snow model performance that would be impossible to identify through individual site studies. We quantified how snow simulation accuracy varies across environmental gradients and identified the elevation and climate conditions where current snow physics representations excel or struggle.

**Methodological Advancement:**
This workflow demonstrates CONFLUENCE's capacity for **automated snow-specific configuration**, **parallel multi-site processing**, and **standardized validation protocols** that enable robust comparative snow science at unprecedented scales.

**Connection to Broader Large Sample Studies:**
Having explored energy balance validation with FLUXNET (04a) and snow processes with NorSWE (04b), we've established the foundation for comprehensive large sample hydrological analysis. The systematic validation approaches developed here extend naturally to basin-scale discharge validation and multi-variable integrated assessments.

The large sample methodology transforms snow hydrology from site-specific case studies to systematic, generalizable science—essential for understanding snow processes in a changing climate.

### Next Focus: Large Sample Experiments - CAMELS-Spat
**Ready to explore large sample basin simulations?** → **[Tutorial 04c: Large Sample Studies - CAMELS-Spat](./04c_large_sample_camelsspat.ipynb)**